In [1]:
import pandas as pd
import os 
data_space = os.path.join(os.getcwd(), "data")
file_24 = "FY24.xls.xlsx"
file_23 = "FY21-23.xlsx"
file_20 = "FY18-20.xlsx"
file_17 = "FY16-17.xlsx"

data_24 = pd.read_excel(os.path.join(data_space,file_24),parse_dates=['Incident Date'])
data_24 = pd.read_excel(os.path.join(data_space, file_24), parse_dates=['Incident Date'])
data_23 = pd.read_excel(os.path.join(data_space, file_23), parse_dates=['Incident Date'])
data_20 = pd.read_excel(os.path.join(data_space, file_20), parse_dates=['Incident Date'])
data_17 = pd.read_excel(os.path.join(data_space, file_17), parse_dates=['Incident Date'])

# Concatenate all dataframes into one
combined_data = pd.concat([data_24, data_23, data_20, data_17], ignore_index=True)
#combined_data = data_24
# Get the minimum date across all the files
min_date = combined_data['Incident Date'].min()

# Fill missing 'Incident Date' values with the minimum date
combined_data['Incident Date'] = combined_data['Incident Date'].fillna(min_date)
print(f"Total Incidents in Study : {len(combined_data)}")

Total Incidents in Study : 8393


/var/folders/1y/msks1tdj7jgb7ghkxzmkq_yh0000gn/T/ipykernel_22523/1766030130.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([data_24, data_23, data_20, data_17], ignore_index=True)


In [2]:
# Filter rows with text values 'A' or 'E'
locations_to_search = ['Sherman','Marshalltown']
def filter_rows_by_location_text_exclude_nan(df, locations):
    """
    Filters rows based on the presence of any specific text values in an array in any columns 
    starting with 'Location', excluding NaN values.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    locations (list of str): The list of text values to search for in the Location columns.

    Returns:
    pd.DataFrame: Filtered DataFrame where any Location columns contain any of the text values.
    """
    # Filter columns that start with 'Location'
    location_columns = [col for col in df.columns if col.startswith('Location')]
    
    # Create a boolean mask for each row where any of the locations match any of the location columns
    mask = df[location_columns].apply(lambda row: row.fillna('').astype(str).apply(
        lambda x: any(loc in x for loc in locations)), axis=1)
    
    # Filter the DataFrame based on the combined mask
    filtered_rows = df[mask.any(axis=1)]
    
    return filtered_rows
#locations_to_search = ['Marshalltown']
filtered_df = filter_rows_by_location_text_exclude_nan(combined_data, locations_to_search)
print(len(combined_data), len(filtered_df))

8393 807


In [8]:
import lancedb
import pandas as pd

def save_dataframes_to_lancedb(df, path, table_name):
    
    # Connect to LanceDB at the given directory (persistent storage)
    ldb = lancedb.connect(path)

    if table_name not in ldb.list_tables():
        ldb.create_table(table_name, data=df.to_dict(orient="records"))
    else:
        print(f"Table '{table_name}' already exists.")

    print(f"DataFrames saved to LanceDB at '{path}' as '{table_name}'.")

def read_table_from_lancedb(path, table_name):
    """
    Reads a table from LanceDB storage.

    :param path: Directory path where LanceDB is stored
    :param table_name: Name of the table to read from LanceDB
    :return: Pandas DataFrame containing the data from LanceDB table
    """
    # Connect to LanceDB
    ldb = lancedb.connect(path)

    # Load the table from LanceDB
    if table_name in ldb.list_tables():
        table = ldb.open_table(table_name)
        # Convert the LanceDB table to a pandas DataFrame
        df = pd.DataFrame(table.to_pandas())
        print(f"Table '{table_name}' loaded successfully from '{path}'.")
        return df
    else:
        print(f"Table '{table_name}' does not exist in LanceDB storage.")
        return None
data_space = os.path.join(os.getcwd(), "data")
data_lance = os.path.join(os.getcwd(), "lance")
save_dataframes_to_lancedb(filtered_df, data_lance, 'incidents_filtered')

#df_incidents = read_table_from_lancedb('/data/lance', 'incidents_filtered')

AttributeError: 'LanceDBConnection' object has no attribute 'list_tables'

In [28]:
import requests
import json
table = db["incidents_filtered"]
import lancedb

def prepare_data(df):
    """
    Helper function to prepare data and return a new DataFrame with 'text' and 'embedding' columns.
    
    :param df: Input pandas DataFrame.
    :param get_embeddings: Function to generate embeddings for a given text.
    
    :return: DataFrame with two columns: 'text' and 'embedding'.
    """
    # Create an empty list to store the text and embeddings
    data = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Convert the row to a dictionary where the column names are keys
        row_dict = {col: str(row[col]) for col in df.columns}
        
        # Flatten the dictionary into a string where each key-value pair is joined by a separator
        flat_text = ', '.join([f"{key}: {value.replace('nan', '')}" for key, value in row_dict.items()])
        
        # Generate the embedding using the provided function
        embedding = get_embeddings(flat_text)
        
        # Append the flat text and embedding as a tuple to the data list
        data.append({'text': flat_text, 'vector': embedding})
    
    # Convert the list of dictionaries into a DataFrame
    result_df = pd.DataFrame(data, columns=['text', 'vector'])
    
    return result_df

#db.create_table("incidents_filtered_embeddings", )       

def get_embeddings(content):
    url = "http://localhost:11434/api/embeddings"
    payload = {
        "model": "nomic-embed-text",
        "prompt": content
    }
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    response.raise_for_status()
    return response.json()["embedding"]
k = 5

result_df = prepare_data(filtered_df)
result_df.head()



,text,vector
0,"Incident Number: AS-IN-20231002-004, Incident ...","[0.6503572463989258, -0.21219654381275177, -2...."
1,"Incident Number: AS-IN-20231004-003, Incident ...","[0.16177916526794434, -0.7963669300079346, -2...."
2,"Incident Number: AS-IN-20231030-001, Incident ...","[0.4862600862979889, -0.06751403957605362, -2...."
3,"Incident Number: AS-IN-20231103-002, Incident ...","[0.6799659132957458, -0.1716877967119217, -2.0..."
4,"Incident Number: AS-IN-20231104-001, Incident ...","[0.5858640074729919, -0.1184571385383606, -2.1..."


In [29]:
result_df.columns
data_space = os.path.join(os.getcwd(), "data")
data_lance = os.path.join(data_space, ".lancedb")
db = lancedb.connect(data_lance)
table = db.create_table("incidents_marshalltown_sherman", result_df)


In [38]:
def RA(messages, model):
    r = requests.post(
        "http://127.0.0.1:11434/api/chat",
        json={"model": model, "messages": messages, "stream": True},
        stream=True
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
        if body.get("done", False):
            message["content"] = output
            return message

In [62]:
data_space = os.path.join(os.getcwd(), "data")
jsa_datafolder =  os.path.join(data_space, "JSA")
file_jsa = "Horizontal Boring Mill.xlsx"
file_jsa = "JSA 300-01 Aerial Lift.xlsx"
#file_jsa = "JSA 300-02 Power Supply REPLACED with 300-07.xlsx"
jsa = pd.read_excel(os.path.join(jsa_datafolder,file_jsa))
from sklearn.impute import SimpleImputer
import numpy as np
nan = SimpleImputer(missing_values=np.nan,strategy="constant",fill_value="")
task = nan.fit_transform(jsa)
task


array([['', '', '', '', '', '', ''],
       ['', '\n\n',
        '1. Activity/Job/Equipment: Aerial Lift Operation                                                          2. Location: Sherman ',
        '',
        '3. Department: Maintenance                                                 4. Preformed By:Otis Brown',
        '5. Job Title: Maintenance                  6. Date Prepared:12/10/2013',
        'J.S.A. NO. 0300-01                                 Rev. 06/15/2018\n PAGE 1 OF 2'],
       ['', 'Risk Rank', 'Important Steps', '', '', '', 'Pictures'],
       ['', 'High 10', '7. TASKS/PROCEDURES', '',
        '8. HAZARDS (UNSAFE ACTS-CONDITIONS (HOW)',
        '  9. CORRECTIVE MEASURES Engineering Controls * Substition * Administrative Controls * PPE',
        ''],
       ['', 'SAFETY', 'Complete operator training.', '',
        'a) Facility Damage, employee injury, operator injury',
        'a) Assure the employee is in an area away from other employees/equipment/hazards and  is

In [63]:
question = f"Find incidents reported which are similar to {task}"

# Embed Question
query_embedding = embedding = get_embeddings(question)
# Semantic Search
result = table.search(query_embedding).limit(10).to_list()
context = [r["text"] for r in result]
for row in context:
    print(row)
    print("*******************")

Incident Number: AS-NE-20220216-002, Incident Date: 2022-02-16 00:00:00, Incident Type(s): Injury, Title: Finger laceration in assembly, Status: Closed, Category: Near Miss, Description: Assembler was preparing a valve for operation testing. Covers were off of the DVC which exposed threads on the junction box. As assembler climbed down ladder, his right ring finger brushed across these threads resulting in a cut. Assembler had just taken gloves off to install small fasteners on the DVC array. He went to medical right away and received treatment. He will go back to medical in the morning for review., Location (Level 2): Emerson, Location (Level 3): Final Control, Location (Level 4): Flow Controls, Location (Level 5): North America, Location (Level 6): USA/CAN/MEX, Location (Level 7): Marshalltown GRN, Location (Level 8): Darrin Nuese, Location (Level 9): Large Value Stream, Location (Level 10): , Location Tag: , Functional Activity: Assembly, Specific Location of Occurrence: Goliath, Di

In [37]:

template = [
            {"Name": "The task being performed at the time of the incident was MOVING PARTS TO FLATS.", 
             "Task Risk": "Without considering the reported incident, the task of moving parts to flats in a manufacturing or industrial setting ...",
             "Task Risk Score":3,
             "Incident Specific Task Risk":"The risk increased due to the use of a rubber mat that insulated the part from the grounded table, which could have allowed for the buildup of static electricity that led to the shock incident",
             "Incident Specific Task Risk Score":5
             }
        ]
question = '1. Identify the Information of the employee referenced in the incident'
envelope = f"You are a friendly AI assistant who finds information for Safety Analysts, analyze the incident information {context} and provide a insightfull response to task or follow the instruction {question}, You are required to respond strictly in JSON format, use the following template {json.dumps(template)}. Do not include any explanation or text outside of the JSON structure."
messages = [
            {"role": "user", "content": envelope}
        ]
response = chatIM(messages,'wizardlm2')
print(response['content'])

['Incident Number: AS-NE-20180209-002, Incident Date: 2018-02-09 00:00:00, Incident Type(s): Injury; Property Damage, Title: Valve fell out of vise, Status: Closed, Category: Near Miss, Description: Employee put a Special Painted 2" 300 RF valve, DJ 15951649 in his vise to do the automation. He had mounted the DVC. Then walked about 12 feet to ask another assembler a question. As he had his back turned to the unit the other assembler seen the valve start to fall and hit a plastic cart that that had another valve staged on it and then go to the floor. This bench is equipped with a jib crane. No one was hurt. The valve had some damage to the flange. The plastic cart had the handle broken and there is a chip in the concreate , Location (Level 2): Emerson, Location (Level 3): Final Control, Location (Level 4): Flow Controls, Location (Level 5): North America, Location (Level 6): USA/CAN/MEX, Location (Level 7): Sherman - FCV, Location (Level 8): Baumann/SSV, Location (Level 9): SSV Assy, L